## Bertalign

`https://github.com/bfsujason/bertalign`

Bertalign is a sentence alignment module for creating parallel corpora and translation memories. It uses multilingual sentence transformer models to align bilingual sentences and documents. Refer to the following paper for details:

`https://academic.oup.com/dsh/article-abstract/38/2/621/6965034`

I've forked the above and made certain modifications to improve sentence tokenization for Chinese. It's available from:

`https://github.com/ruben-tsui/bertalign`

## User inputs: input file names, source and target languages, `max_align` parameter

In [ ]:
#### USER INPUTS ####
# (1) Your source and target input file names must conform to the format:
#     sample.en.txt
#     sample.zh.txt
#     and you will enter the base file name as follows
base = '/content/Ch01'

# (2) Source and target languages (check ISO 689-1 for language codes)
src_lang = 'en'   # source language, e.g. en, zh, es, fr, de, ja, it
tgt_lang = 'zh'   # target language

# (3) Considers all "N sentences-to-M sentences" alignment, N + M <= max_align;
#     The larger this value, the longer it will take to complete the alignment.
max_align = 7

#### DO NOT CHANGE THE FOLLOWING ####
# input files (source and target texts)
fin_src  = f'{base}.{src_lang}.txt'
fin_tgt  = f'{base}.{tgt_lang}.txt'
# output (aligned) file (plain text and Excel)
fon_txt  = f'{base}.bertalign.n{max_align}.{src_lang}-{tgt_lang}.txt'
fon_xlsx = f'{base}.bertalign.n{max_align}.{src_lang}-{tgt_lang}.xlsx'

In [ ]:
!rm -rf bertalign/

In [ ]:
!git clone https://github.com/ruben-tsui/bertalign

Cloning into 'bertalign'...
remote: Enumerating objects: 1174, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1174 (delta 38), reused 29 (delta 29), pack-reused 1123 (from 1)
Receiving objects: 100% (1174/1174), 303.21 MiB | 31.74 MiB/s, done.
Resolving deltas: 100% (435/435), done.


In [ ]:
cd /content/bertalign

/content/bertalign


In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
from bertalign import Bertalign

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/445 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join

In [ ]:
cd ..

/content


## For subsequent text alignments within the same Colab session, modify the first cell (input file names and other parameters) and do a `Runtime -> Run After` in the following cell.

In [ ]:
print(f"Reading source text: {fin_src}...")
src = open(fin_src, 'r', encoding='utf-8').read()
print(f"Reading target text: {fin_tgt}...")
tgt = open(fin_tgt, 'r', encoding='utf-8').read()

Reading source text: /content/Ch01.en.txt...
Reading target text: /content/Ch01.zh.txt...


### Sentence aligning begins below

In [ ]:
%%time
aligner = Bertalign(src, tgt, max_align=max_align, src_lang=src_lang, tgt_lang=tgt_lang)
aligner.align_sents()

Source language: en, Number of sentences: 261
Target language: zh, Number of sentences: 269
Embedding source and target text using LaBSE ...
Performing first-step alignment ...
Performing second-step alignment ...
Finished! Successfully aligning 261 en sentences to 269 zh sentences

CPU times: user 26.7 s, sys: 1.52 s, total: 28.2 s
Wall time: 36.3 s


In [ ]:
from bertalign import model
from bertalign.utils import *
import numpy as np

Creates the sentence-aligned text in .txt (plain text) format

In [ ]:
data = []
with open(fon_txt, 'w', encoding='utf-8', newline='\n') as fo:
    cnt = 0
    for s, t in aligner.result:
        cnt += 1
        # source
        if src_lang in ['zh', 'ja', 'kr']:
            s_delim = ''  # no space between sentences
        else:
            s_delim = ' ' # one space between sentences
        ss = [aligner.src_sents[sidx].strip() for sidx in s]
        ss = s_delim.join(ss)
        sv = model.model.encode(ss, normalize_embeddings=True)
        # target
        if tgt_lang in ['zh', 'ja', 'kr']:
            t_delim = ''  # no space between sentences
        else:
            t_delim = ' ' # one space between sentences
        tt = [aligner.tgt_sents[tidx].strip() for tidx in t]
        tt = t_delim.join(tt)
        tv = model.model.encode(tt, normalize_embeddings=True)
        #
        cossim = np.dot(sv, tv)
        cosdist = 1 - cossim
        line = f"{cosdist:.4f}\t{str(s)}\t{ss}\t{str(t)}\t{tt}\n"
        data.append(line)
        fo.write(line)

Creates the sentence-aligned text in .xlsx (Microsoft Excel) format

In [ ]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils.dataframe import dataframe_to_rows

# Create a new workbook
wb = openpyxl.Workbook()
# Select the active sheet
ws = wb.active
# Set column widths
ws.column_dimensions['A'].width = 10
ws.column_dimensions['B'].width = 10
ws.column_dimensions['C'].width = 10
ws.column_dimensions['D'].width = 50
ws.column_dimensions['E'].width = 10
ws.column_dimensions['F'].width = 65

df = pd.DataFrame([x.split('\t') for x in data], columns=['cosdist', 'cols_s', src_lang, 'cols_t',  tgt_lang])

for r in dataframe_to_rows(df, index=True, header=True):
    ws.append(r)

# Set cell alignment
alignment = Alignment(horizontal='general',
                      vertical='top',
                      wrap_text=True)

for row in ws[f'A1:F{cnt+10}']:
    for cell in row:
        cell.alignment = alignment

# Save the workbook
wb.save(fon_xlsx)
